In [9]:
import sys
sys.version

'3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]'

In [10]:
import platform

In [11]:
platform.python_version()

'3.6.4'

In [22]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urlencode
import urllib3
import string
import json

In [19]:
resp = requests.post("https://csdl-api.computer.org/api/v1/graphql",
                     json={
#                          "operationName":"null",
                         "variables":{"pubType":"journal"},
                         "query":"query ($pubType: String, $idPrefix: String) {\n  periodicals(pubType: $pubType, idPrefix: $idPrefix) {\n    id\n    title\n    idPrefix\n    pubType\n    isOutOfPrint\n    decades\n    yearFrom\n    yearTo\n    subscribeMemberUrl\n    subscribeNonMemberUrl\n    cfpCategoryId\n    featuredArticleId\n    announcementsCategoryId\n    hideSubscribeButton\n    hideLatestArticles\n    currentIssueYear\n    currentIssueNum\n    currentIssue {\n      id\n      idPrefix\n      isPreviewOnly\n      issueNum\n      label\n      pubType\n      title\n      volume\n      year\n      downloadables {\n        issueId\n        hasAzw3\n        hasCover\n        hasEpub\n        hasMathEpub\n        hasMobi\n        hasPdf\n        hasZip\n        __typename\n      }\n      __typename\n    }\n    menuConfig {\n      hasHome\n      homeContentId\n      hasPreprints\n      hideCurrentIssue\n      facebookUrl\n      twitterUrl\n      linkedInUrl\n      rssUrl\n      aboutUs {\n        id\n        name\n        externalUrl\n        postId\n        __typename\n      }\n      writeForUs {\n        id\n        name\n        externalUrl\n        postId\n        __typename\n      }\n      miscLinks {\n        id\n        name\n        externalUrl\n        postId\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"
                     })

In [20]:
body = resp.json()

In [24]:
print(json.dumps(body, sort_keys=True, indent=4))

{
    "data": {
        "periodicals": [
            {
                "__typename": "Periodical",
                "announcementsCategoryId": "0",
                "cfpCategoryId": "0",
                "currentIssue": {
                    "__typename": "PeriodicalIssue",
                    "downloadables": {
                        "__typename": "PeriodicalIssueDownloadablesType",
                        "hasAzw3": false,
                        "hasCover": true,
                        "hasEpub": true,
                        "hasMathEpub": false,
                        "hasMobi": false,
                        "hasPdf": true,
                        "hasZip": false,
                        "issueId": "cq/2018/10"
                    },
                    "id": "12OmNC8uRn0",
                    "idPrefix": "cq",
                    "isPreviewOnly": false,
                    "issueNum": "10",
                    "label": null,
                    "pubType": "journal",
            

title - periodicals - data / id - periodicals - data / 

살펴보면, "data" (key 값) > "periodicals (key 값) > list로 26개의 journals 들어있음. 각각에 대해 "titlet"과 "id" 추출하면 될 듯. 

In [41]:
journals_title = []
journals_id = []

for i in range(0,len(body["data"]['periodicals'])):
    journals_title.append(body["data"]['periodicals'][i]['title'])
    journals_id.append(body["data"]['periodicals'][i]['id'])


다행히 각 저널별 url은 https://www.computer.org/csdl/journal/ca 와 같이 규칙적인 구조를 가짐. 
이 페이지 역시 URL 요청으로 얻어지는 Source Page로는 아무것도 얻을 수 없음. 

In [101]:
resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", # query는 달라져야 함. articleId만 알 수 있으면 Abstract 꺼낼 수 있다. 
                     json={
                         "variables":{"articleId":"13rRUwwslBG"},
                         "query":"query ($articleId: String!) {\n  issue: periodicalIssueByArticleId(articleId: $articleId) {\n    id\n    title\n    year\n    issueNum\n    idPrefix\n    pubType\n    volume\n    year\n    label\n    downloadables {\n      hasCover\n      __typename\n    }\n    __typename\n  }\n  article: articleById(articleId: $articleId) {\n    id\n    doi\n    abstract\n    title\n    fno\n    hasPdf\n    idPrefix\n    keywords\n    authors {\n      givenName\n      surname\n      fullName\n      affiliation\n      __typename\n    }\n    showBuyMe\n    isOpenAccess\n    pubDate\n    pages\n    year\n    __typename\n  }\n  webExtras: webExtrasByArticleId(articleId: $articleId) {\n    id\n    name\n    location\n    extension\n    size\n    __typename\n  }\n}\n"
                     })



In [102]:
body = resp.json()

In [103]:
print(json.dumps(body, sort_keys=True, indent=4))

{
    "data": {
        "article": {
            "__typename": "ArticleType",
            "abstract": "Data confidentiality attacks utilizing memory access patterns threaten exposure of data in modern main memories. Oblivious RAM\u00a0(ORAM) is an effective cryptographic primitive developed to thwart access-pattern-based attacks in DRAM-based systems. However, in emerging non-volatile memory\u00a0(NVM) systems, the increased writes due to encryption of multiple data blocks on every Path ORAM (state-of-the-art efficient ORAM) access impose significant energy, lifetime, and performance overheads. LEO\u00a0(Low overhead E ncryption ORAM)\u00a0is an efficient Path ORAM encryption architecture that addresses the high write overheads of ORAM integration in NVMs, while providing security equivalent to the baseline Path ORAM. LEO reduces NVM cell writes by securely decreasing the number of block encryptions during the write phase of a Path ORAM access. LEO uses a secure, two-level counter mode

In [104]:
resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                     json={
                         "variables":{"announcementsCategoryId":"816","cfpCategoryId":"854","idPrefix":"ca","issueNum":"01","year":"2018"},
                         "query":"query ($idPrefix: String!, $year: String!, $issueNum: String!, $cfpCategoryId: String!, $announcementsCategoryId: String!) {\n  issue: periodicalIssue(idPrefix: $idPrefix, year: $year, issueNum: $issueNum) {\n    id\n    idPrefix\n    isPreviewOnly\n    issueNum\n    pubType\n    downloadables {\n      hasCover\n      hasAzw3\n      hasEpub\n      hasMathEpub\n      hasMobi\n      hasPdf\n      hasZip\n      __typename\n    }\n    title\n    label\n    volume\n    year\n    __typename\n  }\n  articles: articles(idPrefix: $idPrefix, year: $year, issueNum: $issueNum) {\n    id\n    authors {\n      fullName\n      givenName\n      surname\n      __typename\n    }\n    fno\n    isOpenAccess\n    issueNum\n    sectionTitle\n    title\n    year\n    pages\n    pubType\n    idPrefix\n    __typename\n  }\n  trendingArticles: popularArticlesByPeriodical(idPrefix: $idPrefix) {\n    id\n    doi\n    title\n    authors {\n      givenName\n      surname\n      fullName\n      __typename\n    }\n    pages\n    year\n    pubType\n    idPrefix\n    issueNum\n    year\n    fno\n    __typename\n  }\n  announcements: wordpressContent(categoryId: $announcementsCategoryId) {\n    id\n    title\n    url\n    __typename\n  }\n  callForPapers: callForPapers(categoryId: $cfpCategoryId) {\n    id\n    title\n    url\n    __typename\n  }\n}\n"
                     })



In [105]:
body = resp.json()

In [106]:
print(json.dumps(body, sort_keys=True, indent=4))

{
    "data": {
        "announcements": [
            {
                "__typename": "WordpressContentType",
                "id": "27492",
                "title": "IEEE DataPort Announcement",
                "url": "https://www.computer.org/ieee-dataport-announcement"
            }
        ],
        "articles": [
            {
                "__typename": "ArticleType",
                "authors": [
                    {
                        "__typename": "ArticleAuthorType",
                        "fullName": "Alberto Scionti",
                        "givenName": "Alberto",
                        "surname": "Scionti"
                    },
                    {
                        "__typename": "ArticleAuthorType",
                        "fullName": "Somnath Mazumdar",
                        "givenName": "Somnath",
                        "surname": "Mazumdar"
                    },
                    {
                        "__typename": "ArticleAuthorType",
    

In [6]:
url = "https://www.computer.org/csdl/journals"

In [7]:
data = requests.get(url)

In [8]:
bs(data.text, 'html.parser')


<!DOCTYPE html>

<html lang="en">
<head>
<title>CSDL | IEEE Computer Society</title>
<link href="favicon.ico" rel="icon"/>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,maximum-scale=1.0,user-scalable=no" name="viewport"/>
<link href="https://ieeecs-media.computer.org/wp-media/2018/04/27230619/cropped-cs-favicon-512x512-32x32.png" rel="icon" sizes="32x32"/>
<link href="https://www.computer.org/wp-content/themes/starter-theme-master/css/bootstrap.css" rel="stylesheet"/>
<link href="https://www.computer.org/wp-content/themes/starter-theme-master/css/bootstrap-extension.css" rel="stylesheet"/>
<link href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet"/>
<link href="https://d20lqhyll7kadk.cloudfront.net/vendor/fat_zoom/css/zoom.css" rel="stylesheet"/>
<link href="https://d1f50jwmzr9xym.cloudfront.net/csdl-website/assets/css/utilities.css" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Rale

In [4]:
from requests_html import HTMLSession

session = HTMLSession()
url = "https://www.computer.org/csdl/journals"
r = session.get(url)
r.html.render()

parsed = r.content

bs(parsed, 'html.parser')

ModuleNotFoundError: No module named 'requests_html'